In [1]:
#pip install torchvision

In [5]:
import torch 
import torch.autograd as tgrad
from pprint import pprint

In [8]:
# declaring the option price parameters

S = torch.Tensor([80]).requires_grad_()  # stock price
t = torch.Tensor([0]).requires_grad_()   # time period
sigma = torch.Tensor([]).requires_grad_()   # volatility
r = torch.Tensor([0.05]).requires_grad_()   # interest rate
K  = torch.Tensor([70]) # strike price
T = torch.Tensor([1]) # real time

t2m = T-t # time to maturity

d1 = (torch.log(S/K) + (r + 0.5*sigma**2) + t2m)/(sigma*torch.sqrt(t2m))
d2 = d1 - sigma*torch.sqrt(t2m)

N0 =  lambda value: 0.5*(1+torch.erf((value/2**0.5))) # normal distribution function
Nd1 = N0(d1)
Nd2 = N0(d2)

C = S*Nd1 -K*Nd2*torch.exp(-r*t2m)
pprint(C.item())


RuntimeError: a Tensor with 0 elements cannot be converted to Scalar